In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Kolmogorov-Arnold Network Layer Definition
class KANLayer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(KANLayer, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)
        self.activation = nn.ReLU()
        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.hidden(x)
        x = self.activation(x)
        return self.output(x)

# Entropy-Based Sampling for Batch
def entropy_based_sampling_batch(logits):
    probabilities = torch.softmax(logits, dim=-1)
    entropy = -torch.sum(probabilities * torch.log(probabilities + 1e-8), dim=-1)
    return entropy

# Model Combining KAN and Entropix
class EntropixKANModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(EntropixKANModel, self).__init__()
        self.kan_layer = KANLayer(input_dim, hidden_dim, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Process entire batch through the KAN layer
        kan_output = self.kan_layer(x)

        # Apply entropy-based sampling to the entire batch
        entropy = entropy_based_sampling_batch(kan_output)

        # Use the entropy values to weight the final layer's output
        weighted_output = self.output_layer(kan_output) * entropy.unsqueeze(1)
        return weighted_output

# Load and preprocess the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create DataLoaders
train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
test_data = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

# Initialize the model, loss function, and optimizer
model = EntropixKANModel(input_dim=4, hidden_dim=16, output_dim=3)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

# Evaluation loop
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Test Accuracy: {100 * correct / total:.2f}%')

# Train and evaluate the model
train_model(model, train_loader, criterion, optimizer)
evaluate_model(model, test_loader)

Epoch 1, Loss: 1.1563308238983154
Epoch 2, Loss: 1.2336087226867676
Epoch 3, Loss: 0.9677525758743286
Epoch 4, Loss: 0.8561504483222961
Epoch 5, Loss: 0.7402212619781494
Epoch 6, Loss: 0.735029935836792
Epoch 7, Loss: 0.6182444095611572
Epoch 8, Loss: 0.6720882654190063
Epoch 9, Loss: 0.6195396184921265
Epoch 10, Loss: 0.5230082869529724
Test Accuracy: 82.22%


Kolmogorov-Arnold Network + Entropy Based Sampling = ^

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Standard Neural Network Definition
class StandardNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(StandardNN, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.activation = nn.ReLU()
        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.activation(self.layer1(x))
        return self.output(x)

# Load and preprocess the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create DataLoaders
train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
test_data = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

# Initialize the model, loss function, and optimizer
model = StandardNN(input_size=4, hidden_size=16, output_size=3)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}')

# Evaluation loop
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Test Accuracy: {100 * correct / total:.2f}%')

# Train and evaluate the model
train_model(model, train_loader, criterion, optimizer)
evaluate_model(model, test_loader)

Epoch 1, Loss: 1.0459
Epoch 2, Loss: 1.0154
Epoch 3, Loss: 0.9919
Epoch 4, Loss: 0.9706
Epoch 5, Loss: 0.9406
Epoch 6, Loss: 0.9104
Epoch 7, Loss: 0.8802
Epoch 8, Loss: 0.8639
Epoch 9, Loss: 0.8425
Epoch 10, Loss: 0.8142
Test Accuracy: 71.11%


Standard Neural Network = ^